In [ ]:
# -*- coding: utf-8 -*- 

import numpy as np
import pandas as pd

from pathlib import Path

import nibabel as nib

import bids
from bids import BIDSLayout
from tqdm import tqdm

import nilearn as nil
from nilearn.plotting import plot_stat_map, show

import matplotlib.pyplot as plt

In [ ]:
# information for path, hyperparams..
data_dir = '/data2/project_model_based_fmri/ds000005' # custom directory

In [ ]:
bids.config.set_option('extension_initial_dot', True)
layout = BIDSLayout(data_dir, derivatives=True)

In [ ]:
def make_contrasts(design_matrix):
    """ returns a dictionary of four contrasts, given the design matrix"""

    # first generate canonical contrasts
    contrast_matrix = np.eye(design_matrix.shape[1])
    contrasts = dict([(column, contrast_matrix[i])
                      for i, column in enumerate(design_matrix.columns)])

    # Short dictionary of more relevant contrasts
    contrasts = {
        'risky': contrasts['risky'],
        'safe': contrasts['safe'],
        'risky-safe': contrasts['risky'] - contrasts['safe']
    }
    return contrasts

In [ ]:
from nilearn import plotting

def plot_contrast(first_level_model, display_mode='ortho', threshold=2.5):
    """ Given a first model, specify, estimate and plot the main contrasts"""
    
    design_matrix = first_level_model.design_matrices_[0]
    # Call the contrast specification within the function
    contrasts = make_contrasts(design_matrix)
    fig = plt.figure(figsize=(11, 3))
    
    z_maps = []
    # compute the per-contrast z-map
    for index, (contrast_id, contrast_val) in enumerate(contrasts.items()):
        ax = plt.subplot(1, len(contrasts), 1 + index)
        z_map = first_level_model.compute_contrast([contrast_val * len(first_level_model.design_matrices_)], output_type='z_score')

        plotting.plot_stat_map(
            z_map, display_mode=display_mode, threshold=threshold, title=contrast_id, axes=ax)
        z_maps.append(z_map)
    return z_maps

In [1]:
# First level analysis using GLM

In [ ]:
# Single subject, single run

In [ ]:
subject_id = layout.get_subject()[0]
run_num = 1

image_sample = layout.derivatives['fMRIPrep'].get(
    subject=subject_id,
    return_type='file',
    suffix='smoothed',
    run=run_num,
    extension='nii.gz')

event_sample = layout.get(
    subject=subject_id,
    return_type='file',
    suffix='events',
    run=run_num,
    extension='tsv')